### Imports

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Load JSON files into pandas DataFrames with lines=True to handle multiple JSON objects
users_df = pd.read_json('data/users.json', lines=True)
brands_df = pd.read_json('data/brands.json', lines=True)
receipts_df = pd.read_json('data/receipts.json', lines=True)

### 1. Users Data

#### Overview of the Data

In [3]:
# Flatten the nested objects (_id and dates)
users_df['_id'] = users_df['_id'].apply(lambda x: x['$oid'])
users_df['createdDate'] = users_df['createdDate'].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) else pd.NaT)
users_df['lastLogin'] = users_df['lastLogin'].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) else pd.NaT)
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [28]:
print(f"Data types of each column:\n{users_df.dtypes}")
print(f"\nShape: {users_df.shape}")

Data types of each column:
_id                     object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

Shape: (495, 7)


#### Check for Missing Values

In [25]:
missing_values = users_df.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64


1. lastLogin: 62 missing values. This means we don’t have information about the last login time for these users, which may affect understanding their activity status.
2. signUpSource: 48 missing values. Without this, it’s unclear how these users signed up, which could impact marketing or user acquisition analysis.
3. state: 56 missing values. Missing state information may limit geographical analysis, making it harder to understand the distribution of users across regions.

#### Check for Duplicate User IDs

In [27]:
duplicate_ids = users_df['_id'].value_counts()[lambda x: x > 1]
print(f"\nCount of Duplicate User IDs:\n{duplicate_ids}")
print(f"\nTotal Number of Unique User IDs that are Repeated: {len(duplicate_ids)}")


Count of Duplicate User IDs:
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
                            ..
60229990b57b8a12187fe9e0     2
601c2c05969c0b11f7d0b097     2
5ff73b90eb7c7d31ca8a452b     2
5ffcb47d04929111f6e9256c     2
5ff7268eeb7c7d12096da2a9     2
Name: _id, Length: 70, dtype: int64

Total Number of Unique User IDs that are Repeated: 70


There are 70 unique user IDs that are repeated in the dataset, with some IDs appearing up to 20 times. This means that for some users, the same user ID is recorded multiple times, which indicates a significant data duplication issue.

#### Check for Role Inconsistencies

In [7]:
# Check for inconsistencies in 'role' field (should only be 'consumer')
inconsistent_roles = users_df[~users_df['role'].str.lower().isin(['consumer'])]
print(f"\nTotal Records with Role Inconsistencies (Non 'consumer'): {len(inconsistent_roles)}")
print(f"Roles found: {inconsistent_roles['role'].unique()}")


Total Records with Role Inconsistencies (Non 'consumer'): 82
Roles found: ['fetch-staff']


In the dataset, the role field is supposed to only have the value "consumer." However, when checking for inconsistencies, 82 records had a different role value, specifically "fetch-staff."

#### Validate State Abbreviations

In [8]:
import us

# Get a list of valid state abbreviations using the 'us' library
valid_states = [state.abbr for state in us.states.STATES]

# Filter out rows where the state is invalid
invalid_state_rows = users_df[~users_df['state'].isin(valid_states)]

# Get the unique invalid states
unique_invalid_states = invalid_state_rows['state'].unique()

# Print the unique invalid states and the count of rows with invalid state abbreviations
print(f"\nInvalid State Abbreviations: {unique_invalid_states}")
print(f"\nTotal number of rows with invalid states: {len(invalid_state_rows)}")



Invalid State Abbreviations: [nan]

Total number of rows with invalid states: 56


### 2. Brands Data

#### Overview of the Data

In [10]:
print(f"Data types of each column:\n{brands_df.dtypes}")
print(f"\nShape: {brands_df.shape}")

Data types of each column:
_id              object
barcode           int64
category         object
categoryCode     object
cpg              object
name             object
topBrand        float64
brandCode        object
dtype: object

Shape: (1167, 8)


Issue with topBrand Field in brands.json:

While loading brands.json using pandas, the topBrand field, originally encoded as boolean (true/false), was incorrectly interpreted as float64. This happened due to the presence of missing (NaN) values, which pandas represents as floats, causing the entire column to be coerced to float64. 

In [11]:
# Flatten the nested objects (_id and cpg - id, ref)
brands_df['_id'] = brands_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) else x)
brands_df['cpg.$id'] = brands_df['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x else None)
brands_df['cpg.$ref'] = brands_df['cpg'].apply(lambda x: x['$ref'] if isinstance(x, dict) else None)

# Drop the original nested 'cpg' field
brands_df = brands_df.drop(columns=['cpg'])
brands_df.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg.$id,cpg.$ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


#### Check for Missing Values

In [12]:
# Check for missing values (NaN)
missing_values = brands_df.isnull().sum()
print(f"Missing values in each column:\n{missing_values}")

Missing values in each column:
_id               0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
cpg.$id           0
cpg.$ref          0
dtype: int64


Category (155 missing values): Some products do not have a category assigned. This makes it difficult to classify products or analyze them by product type.

CategoryCode (650 missing values): A significant number of products are missing their corresponding categoryCode, which might be necessary for linking products to specific categories.

TopBrand (612 missing values): Many products do not indicate whether they are a top brand. This could affect marketing analysis or promotions for premium products.

BrandCode (234 missing values): Missing brand codes can make it hard to identify the manufacturer or brand associated with a product, leading to confusion when matching products with brands.

#### Check for Duplicates

In [13]:
# Check for duplicate rows in the entire dataset
duplicate_rows = brands_df[brands_df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Check for duplicate '_id' (which should be unique)
duplicate_ids = brands_df[brands_df['_id'].duplicated()]
print(f"Number of duplicate '_id': {len(duplicate_ids)}")

# Check for duplicate 'barcode' (which should also be unique)
duplicate_barcodes = brands_df[brands_df['barcode'].duplicated()]
print(f"Number of duplicate barcodes: {len(duplicate_barcodes)}")

Number of duplicate rows: 0
Number of duplicate '_id': 0
Number of duplicate barcodes: 7


There are 7 duplicate barcodes, which is a problem because barcodes are supposed to uniquely identify products. Having the same barcode for different products can cause confusion and errors in product identification, inventory management, and sales tracking.

#### Check for Inconsistent topBrand Values
The topBrand field, despite being a boolean in the original data, is represented as 0 and 1 in dataset. This is to check if any invalid values (outside of 0 or 1) exist in the dataset.

In [14]:
# Check for invalid 'topBrand' values (anything other than 0 or 1)
invalid_topBrand = brands_df[~brands_df['topBrand'].isin([0, 1])]
print(f"Number of invalid 'topBrand' values: {len(invalid_topBrand)}")
print(invalid_topBrand["topBrand"].unique())

Number of invalid 'topBrand' values: 612
[nan]


#### Check for Invalid Characters in name Field
The name field should not contain any suspicious or invalid patterns (like placeholders or test data)

In [15]:
# Check for suspicious 'name' patterns (e.g., containing the word "test")
suspicious_names = brands_df[brands_df['name'].str.contains('test', case=False, na=False)]
print(f"Number of suspicious brand names: {len(suspicious_names)}")


Number of suspicious brand names: 432


#### Check for Incomplete or Suspicious brandCode Values
brandCode should ideally follow a consistent pattern (e.g., not containing placeholder values like "TEST")

In [16]:
# Check for placeholder or suspicious brandCode patterns (e.g., containing "TEST")
suspicious_brandCodes = brands_df[brands_df['brandCode'].str.contains('TEST', case=False, na=False)]
print(f"Number of suspicious brandCodes: {len(suspicious_brandCodes)}")


Number of suspicious brandCodes: 360


The analysis found 360 instances where the brandCode contains the word "TEST". This suggests the presence of placeholder or test data in the dataset, which may not represent real products.

#### Barcode Consistency Check: Identifying Conflicting Product Information

In [17]:
# Group by 'barcode' and check for conflicting values in other fields
conflicting_barcodes = brands_df.groupby('barcode').filter(lambda x: x.nunique().sum() > len(x.columns))
print(f"Number of barcodes with conflicting information: {len(conflicting_barcodes)}")

# Display conflicting records in a table format
if len(conflicting_barcodes) > 0:
    print("Conflicting barcode records:")
    display(conflicting_barcodes['barcode'].unique())
    display(conflicting_barcodes)  # This will display the data in a table format if you are using Jupyter Notebooks or tools that support DataFrame display
else:
    print("No conflicting barcode records found.")


Number of barcodes with conflicting information: 14
Conflicting barcode records:


array([511111504788, 511111305125, 511111504139, 511111204923,
       511111605058, 511111004790, 511111704140], dtype=int64)

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg.$id,cpg.$ref
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,test,NaN,TEST,59ba6f1ce4b092b29c167346,Cogs
20,5c4699f387ff3577e203ea29,511111305125,Baby,NaN,Chris Image Test,NaN,CHRISIMAGE,55b62995e4b0d8e685c14213,Cogs
129,5a7e0604e4b0aedb3b84afd3,511111504139,Beverages,NaN,Chris Brand XYZ,NaN,CHRISXYZ,55b62995e4b0d8e685c14213,Cogs
152,5c45f91b87ff3552f950f027,511111204923,Grocery,NaN,Brand1,1.0,0987654321,5c45f8b087ff3552f950f026,Cogs
194,5d6415d5a3a018514994f429,511111605058,Magazines,NaN,Health Magazine,NaN,511111605058,5d5d4fd16d5f3b23d1bc7905,Cogs
299,5a8c33f3e4b07f0a2dac8943,511111504139,Grocery,NaN,Pace,0.0,PACE,5a734034e4b0d58f376be874,Cogs
412,5ccb2ece166eb31bbbadccbe,511111504788,Condiments & Sauces,NaN,The Pioneer Woman,NaN,PIONEER WOMAN,559c2234e4b06aca36af13c6,Cogs
467,5c409ab4cd244a3539b84162,511111004790,Baking,NaN,alexa,1.0,ALEXA,55b62995e4b0d8e685c14213,Cogs
536,5d6027f46d5f3b23d1bc7906,511111204923,Snacks,NaN,CHESTER'S,NaN,CHESTERS,5332f5fbe4b03c9a25efd0ba,Cogs
651,5d642d65a3a018514994f42d,511111305125,Magazines,NaN,Rachael Ray Everyday,NaN,511111305125,5d5d4fd16d5f3b23d1bc7905,Cogs


This result shows that there are problems with data accuracy because the same barcode is being used for different products. Barcodes are supposed to uniquely identify one specific product, but in this case, a single barcode is linked to multiple names.

Example:
Barcode 511111504788:
Row 9: Linked to a test product called "test".
Row 412: Linked to a real product called "The Pioneer Woman".
This means the same barcode is used for two different items, which can lead to confusion in identifying products, mistakes in inventory, and errors during sales. It suggests that the data is either incorrect or incomplete.

### 3. Receipts Data

#### Overview of the Data

In [18]:
print(f"Data types of each column:\n{receipts_df.dtypes}")
print(f"\nShape: {receipts_df.shape}")

Data types of each column:
_id                         object
bonusPointsEarned          float64
bonusPointsEarnedReason     object
createDate                  object
dateScanned                 object
finishedDate                object
modifyDate                  object
pointsAwardedDate           object
pointsEarned               float64
purchaseDate                object
purchasedItemCount         float64
rewardsReceiptItemList      object
rewardsReceiptStatus        object
totalSpent                 float64
userId                      object
dtype: object

Shape: (1119, 15)


In [19]:
# Flatten the nested objects 
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
receipts_df['createDate'] = receipts_df['createDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['dateScanned'] = receipts_df['dateScanned'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['finishedDate'] = receipts_df['finishedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['modifyDate'] = receipts_df['modifyDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['pointsAwardedDate'] = receipts_df['pointsAwardedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['purchaseDate'] = receipts_df['purchaseDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)

# pandas option to display floats without scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)

receipts_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.00,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1609687531000.00,1609687536000,1609687531000.00,500.00,1609632000000.00,5.00,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.00,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1609687483000.00,1609687488000,1609687483000.00,150.00,1609601083000.00,2.00,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.00,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.00,1609632000000.00,1.00,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.00,All-receipts receipt bonus,1609687534000,1609687534000,1609687534000.00,1609687539000,1609687534000.00,5.00,1609632000000.00,4.00,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.00,All-receipts receipt bonus,1609687506000,1609687506000,1609687511000.00,1609687511000,1609687506000.00,5.00,1609601106000.00,2.00,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [20]:
# Check for missing values in the DataFrame
missing_values = receipts_df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64


#### Check for Duplicates

In [21]:
# Check for duplicate '_id' fields
duplicate_ids = receipts_df[receipts_df['_id'].duplicated()]
print(f"Number of duplicate '_id': {len(duplicate_ids)}")

Number of duplicate '_id': 0


#### Check for Invalid Dates
Dates should follow a logical order, such as purchaseDate being before dateScanned and pointsAwardedDate.

In [22]:
# Check if 'purchaseDate' is before 'dateScanned'
invalid_dates = receipts_df[receipts_df['purchaseDate'] > receipts_df['dateScanned']]
print(f"Number of records where 'purchaseDate' is after 'dateScanned': {invalid_dates.shape[0]}")

# Check if 'pointsAwardedDate' is after 'purchaseDate'
invalid_points_awarded_date = receipts_df[receipts_df['purchaseDate'] > receipts_df['pointsAwardedDate']]
print(f"Number of records where 'pointsAwardedDate' is before 'purchaseDate': {invalid_points_awarded_date.shape[0]}")


Number of records where 'purchaseDate' is after 'dateScanned': 13
Number of records where 'pointsAwardedDate' is before 'purchaseDate': 7


_13 records where purchaseDate is after dateScanned:_

This means that in 13 instances, receipts were scanned before the purchase took place, which is logically incorrect. These records likely contain data entry or processing errors.

_7 records where pointsAwardedDate is before purchaseDate:_

In these 7 cases, points were awarded before the purchase was made, which is also incorrect. Points should only be awarded after the purchase is completed, so these records reflect a system or data issue.